In [ ]:
#| default_exp detect

In [ ]:
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
#| export
from relax.import_essentials import *
from relax.explain import prepare_cf_module, Explanation
import relax
from scipy.stats import ttest_rel
import einops

In [ ]:
#| export
class DetectionConfig(BaseParser):
    frac: float = 1.0
    tau: float = 0.0

    # validate frac to be within [0, 1]
    def __post_init__(self):
        assert 0 <= self.frac <= 1, f"frac must be a floating value within [0, 1], but got {self.frac}"
        assert 0 <= self.tau <= 1,  f"tau must be a floating value within [0, 1], but got {self.tau}"

In [ ]:
#| export
def detect_watermark(
    cfs: Array,
    w_cfs: Array,
    extracted_model: relax.MLModule,
    config: DetectionConfig | dict,
):
    y_cfs, y_w_cfs = jax.tree_map(lambda x: extracted_model.pred_fn(x), (cfs, w_cfs))
    # reshape and slice
    y_cfs, y_w_cfs = jax.tree_map(lambda x: x[:int(config.frac*len(x))].reshape(-1, 2), 
                                  (y_cfs, y_w_cfs))
    # t-test
    ttest = ttest_rel(
        y_w_cfs[jnp.arange(y_w_cfs.shape[0]), y_w_cfs.argmax(axis=1)] - config.tau,
        y_cfs[jnp.arange(y_w_cfs.shape[0]), y_w_cfs.argmax(axis=1)],
        alternative="greater",
    )

    y_diff = np.mean(
        y_w_cfs[jnp.arange(y_w_cfs.shape[0]), y_w_cfs.argmax(axis=1)]
        - y_cfs[jnp.arange(y_w_cfs.shape[0]), y_w_cfs.argmax(axis=1)]
    )
    return ttest, y_diff

In [ ]:
model = relax.load_ml_module('dummy')
data = relax.load_data('dummy')
cfs = einops.rearrange(data['test'][0], 'b k -> b 1 k')
w_cfs = cfs + jrand.uniform(jrand.PRNGKey(0), shape=cfs.shape, minval=-0.03, maxval=0.03)
res, diff = ttest(cfs, w_cfs, model)